In [57]:
from pathlib import Path
import os, shutil, re, time, json
from typing import Optional, List
from enum import Enum

from dotenv import load_dotenv
from rapidfuzz import fuzz

from pydantic import BaseModel, Field, ValidationError
from langchain.chat_models         import ChatOpenAI
from langchain.embeddings          import OpenAIEmbeddings
from langchain.text_splitter       import CharacterTextSplitter
from langchain.schema              import Document, BaseRetriever
from langchain_community.vectorstores import Chroma
from langchain.retrievers          import BM25Retriever, EnsembleRetriever
from langchain.memory              import ConversationBufferMemory
from langchain.agents              import Tool, AgentType, initialize_agent
from langchain.prompts             import PromptTemplate
from langchain.chains              import LLMChain
from langchain.output_parsers      import PydanticOutputParser, EnumOutputParser
from langchain_core.exceptions     import OutputParserException

import chromadb
from chromadb.config import Settings

In [58]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

persist_dir = str(Path.home() / "chroma_asic_idx")
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

In [59]:
product_texts = [
    """
Bitmain Antminer S19 Pro 110 TH/s
SHA-256 • 110 TH/s • 3250 Вт • 29,5 J/TH
Цена 199 000 ₽ • Гарантия 12 мес.
    """,
    """
MicroBT Whatsminer M30S++ 112 TH/s
SHA-256 • 112 TH/s • 3472 Вт • 31 J/TH
Цена 128 000 ₽ • Гарантия 3 мес.
    """,
    """
iPollo V1 Mini ETC 300 MH/s (Wi-Fi)
EtHash ETC • 300 MH/s • 240 Вт
Цена 38 500 ₽ • Гарантия 6 мес.
    """,
]
documents = [Document(page_content=t.strip()) for t in product_texts]

In [60]:
splitter   = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=150)
chunks     = splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(
    chunks,
    embedding         = embeddings,
    collection_name   = "asic_store",
    persist_directory = persist_dir,
    client_settings   = Settings(anonymized_telemetry=False),
)

In [70]:
bm25 = BM25Retriever.from_documents(documents, k=3)

class FuzzyRetriever(BaseRetriever):
    model_config = {"extra": "allow"}
    _docs: List[Document] = bm25.docs
    _k:   int             = 3

    def _get_relevant_documents(self, query, **_):
        ranked = sorted(
            self._docs,
            key=lambda d: fuzz.partial_ratio(query.lower(), d.page_content.lower()),
            reverse=True,
        )
        return ranked[: self._k]

    async def _aget_relevant_documents(self, query, **_):
        return self._get_relevant_documents(query)

hybrid_retriever = EnsembleRetriever(
    retrievers=[
        vectorstore.as_retriever(search_kwargs={"k": 8}),
        bm25,
        FuzzyRetriever(),
    ],
    weights=[0.5, 0.35, 0.15],
)

In [77]:
class ClientCard(BaseModel):
    name:              Optional[str]  = None
    telegram:          Optional[str]  = None
    phone:             Optional[str]  = None
    location:          Optional[str]  = None
    entity_type:       Optional[str]  = None
    experience:        Optional[int]  = None
    rigs_owned:        Optional[int]  = None
    rigs_plan:         Optional[int]  = None
    electricity_price: Optional[float]= None
    host_choice:       Optional[str]  = None
    free_power:        Optional[int]  = None
    budget:            Optional[int]  = None
    financial_level:   Optional[int]  = None
    knowledge:         Optional[int]  = None
    stage_closed:      Optional[bool] = None

card_parser = PydanticOutputParser(pydantic_object=ClientCard)
card_prompt = PromptTemplate(
    template=(
        "Обнови JSON-карту клиента по новой реплике.\n"
        "Текущий JSON: {cur}\n\n"
        "Реплика: \"{utt}\"\n\n"
        "{fmt}"
    ),
    input_variables=["cur", "utt", "fmt"],
)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
card_chain = LLMChain(llm=llm, prompt=card_prompt, output_parser=card_parser)

def update_card(card: ClientCard, utt: str, retry: int = 2) -> ClientCard:
    for _ in range(retry):
        try:
            raw = card_chain.invoke({
                "cur": card.model_dump_json(),
                "utt": utt,
                "fmt": card_parser.get_format_instructions(),
            })["text"]
            if isinstance(raw, ClientCard):
                return raw
            return ClientCard(**raw) if isinstance(raw, dict) else ClientCard.parse_raw(raw)
        except (OutputParserException, ValidationError):
            time.sleep(0.2)
    return card

In [78]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

class Intent(str, Enum):
    list   = "list"
    detail = "detail"
    budget = "budget"

intent_parser = EnumOutputParser(enum=Intent)

intent_prompt = PromptTemplate(
    template=(
        "Категоризуй запрос клиента одним словом: list, detail или budget.\n"
        "Запрос: \"{q}\"\n\n"
        "{fmt}"
    ),
    input_variables=["q", "fmt"],
)

intent_chain = LLMChain(
    llm=llm,
    prompt=intent_prompt,
    output_parser=intent_parser,
)


In [79]:
def product_info(q: str) -> str:
    intent = intent_chain.invoke({
        "q": q,
        "fmt": intent_parser.get_format_instructions(),
    })["text"].strip()

    if intent == "list":
        return "\n".join(f"• {d.page_content.splitlines()[0]}" for d in documents)

    if intent == "budget":
        m = re.search(r"(\d[\d\s]{3,})", q)
        if not m:
            return "Пожалуйста, уточните бюджет в рублях."
        budget = int(m.group(1).replace(" ", ""))
        fits = [
            d for d in documents
            if (p := re.search(r"Цена\s+(\d[\d\s]+)", d.page_content))
            and int(p.group(1).replace(" ", "")) <= budget
        ]
        return (
            "Подходит:\n" +
            "\n".join(f"• {d.page_content.splitlines()[0]}" for d in fits)
            if fits else "Нет моделей в этом бюджете."
        )

    ql = q.lower()
    for d in documents:
        title = d.page_content.splitlines()[0].lower()
        if any(tok in ql for tok in re.split(r"\W+", title) if len(tok) > 2):
            return d.page_content

    # fallback: семантический поиск
    docs = hybrid_retriever.invoke(q)
    if not docs:
        return "Информация не найдена."
    ctx = "\n---\n".join(d.page_content for d in docs[:2])
    return llm.invoke(
        f"Используя только этот контекст, ответь фактами:\n{ctx}\n\nВопрос: {q}\nОтвет:"
    )


In [80]:
stage_prompt = PromptTemplate.from_template(
    """Определи стадию 1–4.

1 – нужен name ИЛИ location  
2 – выявление потребностей (rigs_owned, rigs_plan, host_choice, electricity_price, free_power, budget)  
3 – презентация решения  
4 – закрытие сделки (сбор контакта и времени)

Правила:
- Слова “купить”, “связаться”, “оформить”, “хочу”, “хотел” → стадия 4.
- Фразы “не интересует”, “просто смотрю” на этапе 2 → сначала краткий pitch.
- Сразу после pitch задаём первый вопрос по блоку 2.

Карта (JSON): {card_json}
История: {chat_history}

Ответь ОДНОЙ цифрой 1–4."""
)
stage_chain = LLMChain(llm=llm, prompt=stage_prompt)

def next_question(card: ClientCard, stage: str) -> Optional[str]:
    if stage == "1":
        if not card.name:
            return "Как мне к вам обращаться?"
        if not card.location:
            return "Где вы планируете размещать оборудование?"
    if stage == "2":
        if card.rigs_owned is None:
            return "Сколько ASIC-майнеров у вас уже есть?"
        if card.rigs_plan is None:
            return "Сколько устройств вы хотели бы приобрести?"
        if not card.host_choice:
            return "Размещать планируете у себя или на нашем хостинге?"
        if card.host_choice and card.host_choice.lower() == "свой" and card.electricity_price is None:
            return "Какая стоимость электроэнергии у вас на площадке (₽/кВт⋅ч)?"
        if card.host_choice and card.host_choice.lower() != "свой" and card.free_power is None:
            return "Сколько свободных кВт вам потребуется на нашем хостинге?"
        if card.budget is None:
            return "Какой бюджет вы закладываете на покупку?"
    if stage == "4":
        if not card.phone and not card.telegram:
            return "Пожалуйста, оставьте телефон или Telegram для созвона."
        return "Когда вам будет удобно созвониться с менеджером?"
    return None

def build_agent(card: ClientCard, stage: str, memory):
    last = memory.buffer[-1].content.lower() if memory.buffer else ""
    pitch = ""
    if stage == "2" and re.search(r"не интересует|просто смотрю", last):
        pitch = (
            "Наше решение снижает затраты на охлаждение до 30 %\n"
            "и повышает надёжность вашей фермы.\n"
        )
    if re.search(r"\b(купить|связаться|оформить|хочу|хотел)\b", last):
        stage = "4"

    q = pitch or next_question(card, stage) or ""

    prefix = f"""
Ты — эксперт по промышленному майнингу.
Этап: {stage}.

{('Питч: ' + pitch) if pitch else ''}
Задача: **задать ОДИН** вопрос или собрать контакт (этап 4):
{q}

Если нужно точное описание модели — TOOL: product_info <вопрос>
Перефразируй Observation, не копируй дословно.
"""
    return initialize_agent(
        tools=[product_tool],
        llm=llm,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        agent_kwargs={"prefix": prefix},
    )


In [81]:
card   = ClientCard()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

print("🟢 Готово! Напишите 'exit' для вывода карточки.")
while True:
    user = input("\nВы: ")
    if user.lower() in {"exit", "quit"}:
        snapshot = card.model_dump(exclude_none=True)
        print("\n📇 Карточка клиента:")
        print(json.dumps(snapshot, indent=2, ensure_ascii=False))
        break

    print(f"👤 Клиент: {user}")
    card = update_card(card, user)
    memory.chat_memory.add_user_message(user)

    stage = stage_chain.invoke({
        "card_json":   card.model_dump_json(),
        "chat_history": memory.buffer,
    })["text"].strip()
    if stage not in {"1","2","3","4"}:
        stage = "1"
    print(f"[Стадия: {stage}]")

    agent = build_agent(card, stage, memory)
    reply = agent.invoke({"input": user})["output"]
    print(f"🤖 Продавец: {reply}")

    card = update_card(card, reply)
    memory.chat_memory.add_ai_message(reply)

🟢 Готово! Напишите 'exit' для вывода карточки.
👤 Клиент: Добрый день
[Стадия: 1]


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: Добрый день! Как мне к вам обращаться?
```

> Finished chain.
🤖 Продавец: Добрый день! Как мне к вам обращаться?
```
👤 Клиент: Иван
[Стадия: 1]


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: Приятно познакомиться, Иван! Где вы планируете устанавливать оборудование для майнинга?
```

> Finished chain.
🤖 Продавец: Приятно познакомиться, Иван! Где вы планируете устанавливать оборудование для майнинга?
```
👤 Клиент: У себя
[Стадия: 2]


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: Понял, у вас будет оборудование на личной территории. Сколько ASIC-майнеров вы уже приобрели?
```

> Finished chain.
🤖 Продавец: Понял, у вас будет оборудование на личной территории. Сколько ASIC-майнеров вы уже приобрели?
```
👤 Клиент: пока нисколько
[Стадия: 2]


> Entering